In [42]:
import requests
import pandas as pd
import html5lib
import io
import numpy as np
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [43]:
# Importing the table of postal codes and transforming it into a dataframe
l1 = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = l1[0]
# Removing cells with a borough that is not assigned
df = df[~df['Borough'].str.contains('Not assigned')]
# Combining rows with the same postal code area but different neighbourhoods
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
# Converting the resulting series into a dataframe
df = pd.DataFrame(df)
# Converting index into a column
df = df.reset_index()
# Replacing not assigned values in neighbourhood column with value in borough column
df['Neighbourhood'].replace('Not assigned', df['Borough'], inplace=True)

In [44]:
df.shape

(103, 3)

In [45]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [46]:
# Importing dataset with postcodes, latitudes and longitudes
df1 = pd.read_csv('http://cocl.us/Geospatial_data')
df1.columns  =['Postcode', 'Latitude', 'Longitude']
# Merging the two dataframes
df_result = pd.merge(df, df1, on='Postcode', how='left')
df_result.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [47]:
df_result['Borough'].unique()
toronto_df = df_result[df_result['Borough'].str.contains('Toronto')]
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [48]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_df[['Latitude', 'Longitude']])

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# adding cluster labels
toronto_df.insert(0, 'Cluster Labels', kmeans.labels_)

In [49]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df
                                  ['Neighbourhood'], toronto_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters